In [46]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [18]:
sales_df = pd.read_csv('sales_01.04.2015.csv')
sales_df.head()

,Unnamed: 0,date,shop_id,item_id,item_price,item_cnt_day
0,2553791,01.04.2015,29,1469,1199.0,1.0
1,2553959,01.04.2015,28,21364,479.0,1.0
2,2553979,01.04.2015,28,21365,999.0,2.0
3,2554011,01.04.2015,28,22104,249.0,2.0
4,2554035,01.04.2015,28,22091,179.0,1.0


In [19]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1515 entries, 0 to 1514
Data columns (total 6 columns):
Unnamed: 0      1515 non-null int64
date            1515 non-null object
shop_id         1515 non-null int64
item_id         1515 non-null int64
item_price      1515 non-null float64
item_cnt_day    1515 non-null float64
dtypes: float64(2), int64(3), object(1)
memory usage: 71.1+ KB


In [24]:
#Date cleaning
#As we can't know if the file is for the 1st of April or the 4th of January, we assume the second.
sales_df['date'] = sales_df['date'].astype('datetime64[ns]')
sales_df.head()

,Unnamed: 0,date,shop_id,item_id,item_price,item_cnt_day
0,2553791,2015-01-04,29,1469,1199.0,1.0
1,2553959,2015-01-04,28,21364,479.0,1.0
2,2553979,2015-01-04,28,21365,999.0,2.0
3,2554011,2015-01-04,28,22104,249.0,2.0
4,2554035,2015-01-04,28,22091,179.0,1.0


In [43]:
#Aggregate per shop
sales_byshop = sales_df.groupby('shop_id').sum().sort_values(by = 'item_cnt_day', ascending = False)

In [33]:
sales_df.describe()

,Unnamed: 0,shop_id,item_id,item_price,item_cnt_day
count,1.515000e+03,1515.000000,1515.000000,1515.000000,1515.000000
mean,2.582539e+06,34.021122,11140.459406,1031.686121,1.103630
std,1.583661e+04,16.569164,6560.093414,2074.376549,0.537086
min,2.553791e+06,2.000000,30.000000,3.000000,-1.000000
25%,2.569732e+06,22.000000,4986.500000,249.000000,1.000000
50%,2.581823e+06,31.000000,11247.000000,479.000000,1.000000
75%,2.595500e+06,50.000000,16633.000000,1191.000000,1.000000
max,2.609988e+06,59.000000,22162.000000,27990.000000,10.000000


In [36]:
#There are negative item counts
#We should ask whether this coresponds to stock corrections or mistakes. In the latter case, we would delete these records.
sales_df[sales_df['item_cnt_day'] == -1]

,Unnamed: 0,date,shop_id,item_id,item_price,item_cnt_day
179,2561299,2015-01-04,35,7877,3990.00,-1.0
305,2567158,2015-01-04,25,2575,2099.00,-1.0
386,2569950,2015-01-04,21,2946,449.00,-1.0
391,2570166,2015-01-04,21,1523,799.00,-1.0
825,2585039,2015-01-04,52,16677,332.67,-1.0
899,2587477,2015-01-04,44,14652,199.00,-1.0
901,2587625,2015-01-04,44,8095,499.00,-1.0
926,2588657,2015-01-04,44,1114,299.00,-1.0
1161,2596614,2015-01-04,42,1878,2599.00,-1.0
1416,2606905,2015-01-04,19,2690,1598.00,-1.0


In [45]:
#Aggregate per item
sales_byitem = sales_df.groupby('item_id').sum().sort_values(by = 'item_cnt_day', ascending = False)

In [47]:
#Connect to DB
driver = 'mysql+pymysql:'
user = 'root'
password = '17028854'
ip = '127.0.0.1'
database = 'Sales_challenge_week3'

connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)

In [50]:
date = pd.to_datetime('today')
sales_df.to_sql(f'CleanSales {date}',connection_string)

In [51]:
sales_byshop.to_sql(f'SalesByShop {date}',connection_string)

In [53]:
sales_byitem.to_sql(f'SalesByItem {date}',connection_string)